In [20]:
from fastapi import FastAPI, Response
from datetime import datetime
from flask import request
import datetime
from urllib.request import urlopen
from PIL import Image,ImageDraw, ImageFont
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import io
from starlette.responses import StreamingResponse
import xarray as xr
import httplib2


In [10]:
Image.open()

TypeError: open() missing 1 required positional argument: 'fp'

In [21]:
def get_stats(minx:float,miny:float,maxx:float,maxy:float,timestamp:str,format:str, scale:int = 1, layer='GHRSST_L4_MUR_Sea_Surface_Temperature', colormap = 'GHRSST_Sea_Surface_Temperature.xml'):

    timestamp = timestamp.replace('-T','T')  # Addreessing timestamp formatting issue

    resolution = 2

    map_height=180*resolution
    map_width=360*resolution

    ###wms_url = 'https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&BBOX=-90,-180,90,180&CRS=EPSG:4326&WIDTH='+str(map_width)+'&HEIGHT='+str(map_height)+'&LAYERS=MODIS_AQUA_L3_Sea_Ice_Daily&STYLES=&FORMAT=image/png&TIME='+timestamp
    wms_url = f'https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&BBOX=-90,-180,90,180&CRS=EPSG:4326&WIDTH='+str(map_width)+'&HEIGHT='+str(map_height)+f'&LAYERS={layer}&STYLES=&FORMAT=image/png&TIME='+timestamp
    print (wms_url)
    img_bytes = urlopen(wms_url).read()
    im = Image.open(requests.get(wms_url, stream=True).raw)
    
    data = np.array(im)

    ###colormap_url = 'https://gibs.earthdata.nasa.gov/colormaps/v1.3/MODIS_Sea_Ice_Daily.xml'
    colormap_url= f'https://gibs.earthdata.nasa.gov/colormaps/v1.3/{colormap.replace(".xml","")}.xml'

    #colormap_url = f'https://gibs.earthdata.nasa.gov/colormaps/v1.3/GHRSST_Sea_Surface_Temperature.xml'

    colormap_document = requests.get(colormap_url)
    colormap_document_soup= BeautifulSoup(colormap_document.content,"lxml-xml")

    colormap_entries = colormap_document_soup.find_all("ColorMapEntry",{"value": True})
    color_entry_dict={}
    for row in colormap_entries:
        color_entry={tuple(map(int, (row.attrs['rgb']+',255').split(","))):float(row.attrs['value'][1:-1].split(',')[0].replace("INF", "0"))}
        color_entry_dict.update(color_entry)

    # pixels = im.load() # create the pixel map
    pixels = np.asarray(im)
    out = np.zeros((im.size[1], im.size[0]))
    for i in range(im.size[0]): # for every pixel:
        for j in range(im.size[1]):
        #out[j-1,i-1] = getValueFromColor(pixels[j,i],color_entry_dict)
            try:
                out[j-1,i-1] = float(color_entry_dict[tuple(pixels[i,j])]) * scale
            except:
                out[j-1,i-1] = None 
    out_pandas = pd.DataFrame(out)
    minx_grid = int((180 + minx)*resolution)
    maxx_grid = int((180 + maxx)*resolution)
    miny_grid = int((90-miny)*resolution)
    maxy_grid = int((90-maxy)*resolution)

    mean_value = np.nanmean(out_pandas[maxy_grid:miny_grid].iloc[:, minx_grid:maxx_grid].values).round(2)
    min_value = np.nanmin(out_pandas[maxy_grid:miny_grid].iloc[:, minx_grid:maxx_grid].values).round(2)
    max_value = np.nanmax(out_pandas[maxy_grid:miny_grid].iloc[:, minx_grid:maxx_grid].values).round(2)
    stdev_value = np.nanstd(out_pandas[maxy_grid:miny_grid].iloc[:, minx_grid:maxx_grid].values).round(2)


    msg = {
        "Mean": mean_value,
        "Max": max_value,
        "Min": min_value,
        "StDev": stdev_value
    }

    stats = {}
    stats["mean"] = str(mean_value)
    stats["max"] = str(max_value)
    stats["min"] = str(min_value)
    stats["stdev"] = str(stdev_value)
    return stats

In [13]:
minx=-160
miny=-5
maxx=-150
maxy=0
timestamp='2016-06-09T00:00:00Z'
format_='json'
layer='MERRA2_2m_Air_Temperature_Assimilated_Monthly'
colormap='MERRA2_2m_Air_Temperature_Assimilated_Monthly.xml'
scale = 1
minx, miny,maxx,maxy, timestamp,format_, scale, layer, colormap

(-160,
 -5,
 -150,
 0,
 '2016-06-09T00:00:00Z',
 'json',
 1,
 'MERRA2_2m_Air_Temperature_Assimilated_Monthly',
 'MERRA2_2m_Air_Temperature_Assimilated_Monthly.xml')

miny=-5
maxx=-150
maxy=0
timestamp='2016-06-09T00:00:00Z'
format_='json'
layer='MOPITT_CO_Daily_Surface_Mixing_Ratio_Day'
colormap='MOPITT_CO_Daily_Surface_Mixing_Ratio_Day.xml'
scale = 1
get_stats(minx, miny,maxx,maxy, timestamp,format_, scale, layer, colormap)

In [14]:
minx=-160
miny=-5
maxx=-150
maxy=0
timestamp='2016-06-09T00:00:00Z'
format_='json'
layer='GHRSST_L4_MUR_Sea_Surface_Temperature'
colormap = 'GHRSST_Sea_Surface_Temperature.xml'
scale = 1
ignore_zeroes = False
get_stats(minx, miny,maxx,maxy, timestamp,format_, scale, layer, colormap)

https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&BBOX=-90,-180,90,180&CRS=EPSG:4326&WIDTH=720&HEIGHT=360&LAYERS=GHRSST_L4_MUR_Sea_Surface_Temperature&STYLES=&FORMAT=image/png&TIME=2016-06-09T00:00:00Z


{'mean': '0.32', 'max': '2.1', 'min': '-0.0', 'stdev': '0.54'}

In [15]:
minx=-160
miny=-5
maxx=-150
maxy=0
timestamp='2016-06-09T00:00:00Z'
format_='json'
layer='MOPITT_CO_Monthly_Total_Column_Day'
colormap = 'MOPITT_CO_Monthly_Total_Column_Day.xml'
scale = 1
ignore_zeroes = False
get_stats(minx, miny,maxx,maxy, timestamp,format_, scale, layer, colormap)

https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&BBOX=-90,-180,90,180&CRS=EPSG:4326&WIDTH=720&HEIGHT=360&LAYERS=MOPITT_CO_Monthly_Total_Column_Day&STYLES=&FORMAT=image/png&TIME=2016-06-09T00:00:00Z


{'mean': '1.7841249999999997e+18',
 'max': '2.1875e+18',
 'min': '1.6875e+18',
 'stdev': '7.87074289695706e+16'}

In [16]:
minx=-160
miny=-5
maxx=-150
maxy=0
timestamp='2016-06-09T00:00:00Z'
format_='json'
layer='GHRSST_L4_MUR25_Sea_Ice_Concentration'
colormap = 'GHRSST_Sea_Ice_Concentration.xml'
scale = 1
ignore_zeroes = False
get_stats(minx, miny,maxx,maxy, timestamp,format_, scale, layer, colormap)

https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&BBOX=-90,-180,90,180&CRS=EPSG:4326&WIDTH=720&HEIGHT=360&LAYERS=GHRSST_L4_MUR25_Sea_Ice_Concentration&STYLES=&FORMAT=image/png&TIME=2016-06-09T00:00:00Z


{'mean': '72.5', 'max': '100.0', 'min': '7.0', 'stdev': '25.29'}

In [17]:
minx=-160
miny=-5
maxx=-150
maxy=0
timestamp='2016-06-09T00:00:00Z'
format_='json'
layer='AIRS_L2_Cloud_Top_Height_Day'
colormap = 'AIRS_Cloud_Top_Height.xml'
scale = 1
ignore_zeroes = False
get_stats(minx, miny,maxx,maxy, timestamp,format_, scale, layer, colormap)

https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&BBOX=-90,-180,90,180&CRS=EPSG:4326&WIDTH=720&HEIGHT=360&LAYERS=AIRS_L2_Cloud_Top_Height_Day&STYLES=&FORMAT=image/png&TIME=2016-06-09T00:00:00Z


{'mean': '4.62', 'max': '10.38', 'min': '1.02', 'stdev': '2.62'}

In [18]:
#same as last but with a different date
minx=-160
miny=-5
maxx=-150
maxy=0
timestamp='2015-02-09T00:00:00Z'
format_='json'
layer='AIRS_L2_Cloud_Top_Height_Day'
colormap = 'AIRS_Cloud_Top_Height.xml'
scale = 1
ignore_zeroes = False
get_stats(minx, miny,maxx,maxy, timestamp,format_, scale, layer, colormap)

https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&BBOX=-90,-180,90,180&CRS=EPSG:4326&WIDTH=720&HEIGHT=360&LAYERS=AIRS_L2_Cloud_Top_Height_Day&STYLES=&FORMAT=image/png&TIME=2015-02-09T00:00:00Z


{'mean': '2.88', 'max': '5.22', 'min': '0.66', 'stdev': '1.0'}

In [43]:
import subprocess
subprocess.call(['ls','-a'])

.
..
api2.py
api3.py
api_pre.py
api.py
foobar.py
get_stats.ipynb
__init__.py
.ipynb_checkpoints
main.py
__pycache__
Roboto-Regular.ttf
spameggs.py
test.py
Untitled1.ipynb
Untitled2.ipynb
Untitled3.ipynb
Untitled4.ipynb
Untitled5.ipynb
Untitled.ipynb


0

In [19]:
import subprocess
minx=-160
miny=-5
maxx=-150
maxy=0
timestamp='2015-02-09T00:00:00Z'
format_='json'
layer='AIRS_L2_Cloud_Top_Height_Day'
colormap = 'AIRS_Cloud_Top_Height.xml'
scale = 1
subprocess.call(['python', 'foobar.py', str(minx), str(miny), str(maxx), str(maxy), str(timestamp), str(scale), str(layer), str(colormap)])

/Users/vaish/Documents/arena/backend/bin/python: can't open file '/Users/vaish/Documents/arena/vaish_podaac_sep23/foobar.py': [Errno 2] No such file or directory


2